# 0. Import packages and data

In [2]:
import pandas as pd
import datetime

In [3]:
# Read csv files

# Day ahead prices
day_ahead_df = pd.read_csv('.\Data\Initial\day_ahead_prices_full.csv', sep=';', index_col=0)

# Generation per unit
generation_df = pd.read_csv('.\Data\Initial\generation_full.csv', sep=';', header=0, index_col=0, low_memory=False)

# Generation per production type
production_type_df = pd.read_csv('.\Data\Initial\production_type_full.csv', sep=';', header=0, index_col=0, low_memory=False)

# Load
load_df = pd.read_csv('.\Data\Initial\load_full.csv', sep=';', header=0, index_col=0)

# Scheduled exchanges
exchanges_DE_BE_df = pd.read_csv('.\Data\Initial\scheduled_exchanges_DE_LU_BE_full.csv', sep=';', index_col=0)
exchanges_FR_BE_df = pd.read_csv('.\Data\Initial\scheduled_exchanges_FR_BE_full.csv', sep=';', index_col=0)
exchanges_NL_BE_df = pd.read_csv('.\Data\Initial\scheduled_exchanges_NL_BE_full.csv', sep=';', index_col=0)
exchanges_GB_BE_df = pd.read_csv('.\Data\Initial\scheduled_exchanges_GB_BE_full.csv', sep=';', index_col=0)

# Outages
outages_df = pd.read_csv(r'.\Data\Initial\outages_full.csv', sep=';', index_col=0)

# Gas prices
gas_prices_df = pd.read_csv('.\Data\Initial\gas_prices.csv', sep=',', header=0)

# 1. Initial Data Preparation

## Day ahead prices

In [3]:
# Inspect
day_ahead_df.head(5)

,Unnamed: 0,day_ahead_price
0,2022-01-01 00:00:00+01:00,82.02
1,2022-01-01 01:00:00+01:00,67.07
2,2022-01-01 02:00:00+01:00,75.11
3,2022-01-01 03:00:00+01:00,50.91
4,2022-01-01 04:00:00+01:00,37.67


In [4]:
# Check shape
print(day_ahead_df.shape)

(52584, 2)


In [5]:
# Rename date column
day_ahead_df.rename(columns={'Unnamed: 0':'date'}, inplace=True )

# Convert date column to datetime type and set time in CET
day_ahead_df['date'] = pd.to_datetime(day_ahead_df['date'], utc=True).dt.tz_convert("CET")

In [6]:
# Set date as index
day_ahead_df.set_index('date', inplace=True)

In [7]:
# Sort index from old to recent
day_ahead_df = day_ahead_df.sort_index()

In [8]:
# Check data types
print(day_ahead_df.dtypes)

day_ahead_price    float64
dtype: object


In [9]:
# Check
day_ahead_df.head(5)

,day_ahead_price
date,
2017-01-01 00:00:00+01:00,42.00
2017-01-01 01:00:00+01:00,49.94
2017-01-01 02:00:00+01:00,51.95
2017-01-01 03:00:00+01:00,40.99
2017-01-01 04:00:00+01:00,39.00


- **date**: date and time regarding the price (covers the whole next hour), expressed in CET
- **day_ahead_price**: electricity price during the hour, in €/MWh

## Gas prices

In [10]:
# Inspect
gas_prices_df.head(5)

,Date,Price,Open,High,Low,Vol.,Change %
0,12/30/2022,76.315,76.315,76.315,76.315,NaN,-8.96%
1,12/29/2022,83.830,83.830,83.830,83.830,NaN,3.05%
2,12/28/2022,81.350,81.350,81.350,81.350,NaN,1.63%
3,12/27/2022,80.045,80.045,80.045,80.045,NaN,-3.53%
4,12/23/2022,82.975,82.975,82.975,82.975,NaN,-9.75%


In [11]:
# Check shape
print(gas_prices_df.shape)

(1311, 7)


In [12]:
# Only keep Date and Price column
gas_prices_df = gas_prices_df.iloc[:,0:2]

In [13]:
# Rename columns
gas_prices_df.rename(columns={'Date':'date'}, inplace=True)
gas_prices_df.rename(columns={'Price':'gas_price'}, inplace=True)

In [14]:
# Convert date column to datetime type and set time in CET
gas_prices_df['date'] = pd.to_datetime(gas_prices_df['date'], utc=True).dt.tz_convert("CET")

In [15]:
# Set date as index
gas_prices_df.set_index('date', inplace=True)

In [16]:
# Set index around midnight
gas_prices_df.index = gas_prices_df.index.normalize()

In [17]:
# Sort index from old to recent
gas_prices_df = gas_prices_df.sort_index()

In [18]:
# Check data types
print(gas_prices_df.dtypes)

gas_price    float64
dtype: object


In [19]:
# Check
gas_prices_df.head(10)

,gas_price
date,
2017-10-23 00:00:00+02:00,18.090
2017-10-24 00:00:00+02:00,17.960
2017-10-25 00:00:00+02:00,18.110
2017-10-26 00:00:00+02:00,18.070
2017-10-27 00:00:00+02:00,18.150
2017-10-30 00:00:00+01:00,18.110
2017-10-31 00:00:00+01:00,18.285
2017-11-01 00:00:00+01:00,18.550
2017-11-02 00:00:00+01:00,18.580


- **date**: date and time regarding the price (covers the whole next hour), expressed in CET
- **gas_price**: gas price during the day, in €/MWh

## Join daily day ahead prices and gas prices

In [20]:
# Resample day ahead price per day
daily_day_ahead_df = day_ahead_df.resample('D').mean()

In [21]:
# Inspect
daily_day_ahead_df.head(5)

,day_ahead_price
date,
2017-01-01 00:00:00+01:00,44.092917
2017-01-02 00:00:00+01:00,54.861250
2017-01-03 00:00:00+01:00,62.062500
2017-01-04 00:00:00+01:00,55.961250
2017-01-05 00:00:00+01:00,58.370000


In [22]:
# Left outer join of daily day ahead and gas prices
daily_prices_df = daily_day_ahead_df.join(gas_prices_df, how="left")

In [23]:
# Fill weekend values with value from Friday, so forward fill
daily_prices_df = daily_prices_df.fillna(method='ffill')

In [24]:
# Drop rows with NaN
daily_prices_df.dropna(inplace=True)

In [25]:
# Check
daily_prices_df

,day_ahead_price,gas_price
date,,
2017-10-23 00:00:00+02:00,65.805000,18.090
2017-10-24 00:00:00+02:00,52.650000,17.960
2017-10-25 00:00:00+02:00,58.244167,18.110
2017-10-26 00:00:00+02:00,64.522917,18.070
2017-10-27 00:00:00+02:00,53.442083,18.150
...,...,...
2022-12-27 00:00:00+01:00,119.485000,80.045
2022-12-28 00:00:00+01:00,51.046250,81.350
2022-12-29 00:00:00+01:00,33.221667,83.830


## Generation per generation unit

In [26]:
# Inspect
generation_df.head(5)

,Unnamed: 0,Amercoeur 1 R GT,Amercoeur 1 R ST,Belwind Phase 1,COO 1 T,COO 2 T,COO 3 T,COO 4 T,COO 5 T,COO 6 T,...,TIHANGE 2,TIHANGE 3,Thorntonbank - C-Power - Area NE,Thorntonbank - C-Power - Area SW,VILVOORDE GT,VILVOORDE ST,Zandvliet Power,Zelzate 2 Knippegroen,LANGERLO 1,LANGERLO 2
0,NaN,Fossil Gas,Fossil Gas,Wind Offshore,Hydro Pumped Storage,Hydro Pumped Storage,Hydro Pumped Storage,Hydro Pumped Storage,Hydro Pumped Storage,Hydro Pumped Storage,...,Nuclear,Nuclear,Wind Offshore,Wind Offshore,Fossil Gas,Fossil Gas,Fossil Gas,Fossil Gas,NaN,NaN
1,2022-01-01 00:00:00+01:00,100.0,94.0,114.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1001.0,1033.0,104.0,128.0,0.0,0.0,0.0,302.0,NaN,NaN
2,2022-01-01 01:00:00+01:00,150.0,110.0,82.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1002.0,1033.0,58.0,85.0,0.0,0.0,0.0,302.0,NaN,NaN
3,2022-01-01 02:00:00+01:00,123.0,101.0,83.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1002.0,1034.0,56.0,101.0,0.0,0.0,0.0,299.0,NaN,NaN
4,2022-01-01 03:00:00+01:00,95.0,93.0,71.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1002.0,1033.0,67.0,126.0,0.0,0.0,0.0,301.0,NaN,NaN


In [27]:
# Check shape
print(generation_df.shape)

(52585, 50)


In [28]:
# Check columns
print(generation_df.iloc[0, 1:])

Amercoeur 1 R GT                              Fossil Gas
Amercoeur 1 R ST                              Fossil Gas
Belwind Phase 1                            Wind Offshore
COO 1 T                             Hydro Pumped Storage
COO 2 T                             Hydro Pumped Storage
COO 3 T                             Hydro Pumped Storage
COO 4 T                             Hydro Pumped Storage
COO 5 T                             Hydro Pumped Storage
COO 6 T                             Hydro Pumped Storage
DOEL 1                                           Nuclear
DOEL 2                                           Nuclear
DOEL 3                                           Nuclear
DOEL 4                                           Nuclear
DROGENBOS GT1                                 Fossil Gas
DROGENBOS GT2                                 Fossil Gas
DROGENBOS ST                                  Fossil Gas
EDF Luminus Seraing GT1                       Fossil Gas
EDF Luminus Seraing GT2        

In [29]:
# Get column list for each generation type
fossil_gas_cols = generation_df.columns[generation_df.iloc[0].eq('Fossil Gas')].tolist()
hydro_pumped_cols = generation_df.columns[generation_df.iloc[0].eq('Hydro Pumped Storage')].tolist()
wind_offshore_cols = generation_df.columns[generation_df.iloc[0].eq('Wind Offshore')].tolist()
nuclear_cols = generation_df.columns[generation_df.iloc[0].eq('Nuclear')].tolist()
biomass_cols = generation_df.columns[generation_df.iloc[0].eq('Biomass')].tolist()

In [30]:
# Check generation type lists
print(fossil_gas_cols)
print(hydro_pumped_cols)
print(wind_offshore_cols)
print(nuclear_cols)
print(biomass_cols)

['Amercoeur 1 R GT', 'Amercoeur 1 R ST', 'DROGENBOS GT1', 'DROGENBOS GT2', 'DROGENBOS ST', 'EDF Luminus Seraing GT1', 'EDF Luminus Seraing GT2', 'HERDERSBRUG GT1', 'HERDERSBRUG GT2', 'HERDERSBRUG ST', 'INESCO WKK', 'Marcinelle Energie (Carsid)', 'RINGVAART STEG', 'SAINT-GHISLAIN STEG', 'SERAING TV', 'Scheldelaan Exxonmobil', 'T-power Beringen', 'VILVOORDE GT', 'VILVOORDE ST', 'Zandvliet Power', 'Zelzate 2 Knippegroen']
['COO 1 T', 'COO 2 T', 'COO 3 T', 'COO 4 T', 'COO 5 T', 'COO 6 T', 'PLATE-TAILLE T']
['Belwind Phase 1', 'Mermaid Offshore WP', 'Nobelwind Offshore Windpark', 'Norther Offshore WP', 'Northwester 2', 'Northwind', 'Rentel Offshore WP', 'Seastar Offshore WP', 'Thorntonbank - C-Power - Area NE', 'Thorntonbank - C-Power - Area SW']
['DOEL 1', 'DOEL 2', 'DOEL 3', 'DOEL 4', 'TIHANGE 1N', 'TIHANGE 1S', 'TIHANGE 2', 'TIHANGE 3']
['RODENHUIZE 4']


In [31]:
# Drop first row
generation_df = generation_df.tail(-1)

In [32]:
# Drop last 2 columns (Langerlo)
generation_df.drop(columns=generation_df.columns[-2:], axis=1, inplace=True)

The electricity generation unit in Langerlo was supposed to convert to biomass for electricity generation. However, the project was shut down in 2017.

In [33]:
# Rename date column
generation_df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

# Convert date column to datetime type and set time in CET
generation_df['date'] = pd.to_datetime(generation_df['date'], utc=True).dt.tz_convert("CET")

In [34]:
# Set date as index
generation_df.set_index('date', inplace=True)

In [35]:
# Sort index
generation_df = generation_df.sort_index()

In [36]:
# Convert other columns to float
cols = generation_df.columns
generation_df[cols] = generation_df[cols].apply(pd.to_numeric, errors='coerce')

In [37]:
# Check data types
print(generation_df.dtypes)

Amercoeur 1 R GT                    float64
Amercoeur 1 R ST                    float64
Belwind Phase 1                     float64
COO 1 T                             float64
COO 2 T                             float64
COO 3 T                             float64
COO 4 T                             float64
COO 5 T                             float64
COO 6 T                             float64
DOEL 1                              float64
DOEL 2                              float64
DOEL 3                              float64
DOEL 4                              float64
DROGENBOS GT1                       float64
DROGENBOS GT2                       float64
DROGENBOS ST                        float64
EDF Luminus Seraing GT1             float64
EDF Luminus Seraing GT2             float64
HERDERSBRUG GT1                     float64
HERDERSBRUG GT2                     float64
HERDERSBRUG ST                      float64
INESCO WKK                          float64
Marcinelle Energie (Carsid)     

In [38]:
# Check
generation_df.head(5)

,Amercoeur 1 R GT,Amercoeur 1 R ST,Belwind Phase 1,COO 1 T,COO 2 T,COO 3 T,COO 4 T,COO 5 T,COO 6 T,DOEL 1,...,TIHANGE 1N,TIHANGE 1S,TIHANGE 2,TIHANGE 3,Thorntonbank - C-Power - Area NE,Thorntonbank - C-Power - Area SW,VILVOORDE GT,VILVOORDE ST,Zandvliet Power,Zelzate 2 Knippegroen
date,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00+01:00,193.0,127.0,115.0,0.0,0.0,0.0,0.0,0.0,0.0,432.0,...,0.0,0.0,1025.0,1044.0,41.0,78.0,NaN,NaN,112.0,275.0
2017-01-01 01:00:00+01:00,230.0,139.0,111.0,0.0,0.0,89.0,138.0,0.0,0.0,432.0,...,0.0,0.0,1025.0,1045.0,44.0,82.0,NaN,NaN,0.0,252.0
2017-01-01 02:00:00+01:00,209.0,133.0,114.0,0.0,0.0,0.0,33.0,0.0,0.0,432.0,...,0.0,0.0,1025.0,1045.0,48.0,82.0,NaN,NaN,0.0,277.0
2017-01-01 03:00:00+01:00,193.0,125.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,432.0,...,0.0,0.0,1024.0,1045.0,53.0,75.0,NaN,NaN,0.0,299.0
2017-01-01 04:00:00+01:00,212.0,133.0,131.0,0.0,0.0,0.0,0.0,0.0,0.0,432.0,...,0.0,0.0,1025.0,1045.0,66.0,105.0,NaN,NaN,0.0,302.0


- **date**: date and time of generation (covers the next hour)
- **generation**: generation at a specific time (MW) by a generation unit (identified as generation unit name)

## Generation per production type

In [39]:
# Inspect
production_type_df.head(5)

,Unnamed: 0,Biomass,Fossil Gas,Fossil Oil,Hydro Pumped Storage,Hydro Pumped Storage.1,Hydro Run-of-river and poundage,Nuclear,Other,Solar,Waste,Wind Offshore,Wind Offshore.1,Wind Onshore,Fossil Hard coal
0,NaN,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Consumption,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Aggregated,Actual Consumption,Actual Aggregated,NaN
1,2022-01-01 00:00:00+01:00,280.0,1009.0,0.0,NaN,566.0,6.0,5965.0,437.0,0.0,251.0,1841.0,NaN,464.0,NaN
2,2022-01-01 01:00:00+01:00,224.0,1076.0,0.0,NaN,895.0,6.0,5967.0,456.0,0.0,252.0,1454.0,NaN,408.0,NaN
3,2022-01-01 02:00:00+01:00,213.0,1036.0,0.0,NaN,892.0,5.0,5968.0,477.0,0.0,255.0,1424.0,NaN,339.0,NaN
4,2022-01-01 03:00:00+01:00,223.0,1002.0,0.0,NaN,969.0,5.0,5968.0,463.0,0.0,257.0,1451.0,NaN,423.0,NaN


In [40]:
# Check shape
print(production_type_df.shape)

(52585, 15)


In [41]:
# Drop first row
production_type_df = production_type_df.tail(-1)

In [42]:
# Rename date column
production_type_df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

# Convert date column to datetime type and set time in CET
production_type_df['date'] = pd.to_datetime(production_type_df['date'], utc=True).dt.tz_convert("CET")

In [43]:
# Set date as index
production_type_df.set_index('date', inplace=True)

In [44]:
# Sort index from old to recent
production_type_df = production_type_df.sort_index()

In [45]:
# Convert other columns to float
cols = production_type_df.columns
production_type_df[cols] = production_type_df[cols].apply(pd.to_numeric, errors='coerce')

In [46]:
# Check data types
print(production_type_df.dtypes)

Biomass                            float64
Fossil Gas                         float64
Fossil Oil                         float64
Hydro Pumped Storage               float64
Hydro Pumped Storage.1             float64
Hydro Run-of-river and poundage    float64
Nuclear                            float64
Other                              float64
Solar                              float64
Waste                              float64
Wind Offshore                      float64
Wind Offshore.1                    float64
Wind Onshore                       float64
Fossil Hard coal                   float64
dtype: object


In [47]:
# Check
production_type_df.head(5)

,Biomass,Fossil Gas,Fossil Oil,Hydro Pumped Storage,Hydro Pumped Storage.1,Hydro Run-of-river and poundage,Nuclear,Other,Solar,Waste,Wind Offshore,Wind Offshore.1,Wind Onshore,Fossil Hard coal
date,,,,,,,,,,,,,,
2017-01-01 00:00:00+01:00,354.0,2462.0,0.0,NaN,278.0,8.0,5003.0,652.0,0.0,267.0,337.0,NaN,108.0,0.0
2017-01-01 01:00:00+01:00,354.0,2334.0,0.0,226.0,NaN,8.0,5003.0,645.0,0.0,260.0,345.0,NaN,107.0,0.0
2017-01-01 02:00:00+01:00,355.0,2197.0,0.0,33.0,71.0,8.0,5002.0,660.0,0.0,265.0,364.0,NaN,90.0,0.0
2017-01-01 03:00:00+01:00,354.0,2249.0,0.0,NaN,456.0,8.0,5003.0,656.0,0.0,264.0,342.0,NaN,87.0,0.0
2017-01-01 04:00:00+01:00,358.0,1952.0,0.0,NaN,828.0,8.0,5003.0,651.0,0.0,266.0,439.0,NaN,84.0,0.0


- **date**: date and time of generation (covers the next hour)
- **generation**: generation at a specific time (MW) by a production type (identified as production type name)

## Load

In [4]:
# Inspect
load_df.head(5)

,Unnamed: 0,Actual Load
0,2022-01-01 00:00:00+01:00,8300.0
1,2022-01-01 00:15:00+01:00,8210.0
2,2022-01-01 00:30:00+01:00,8020.0
3,2022-01-01 00:45:00+01:00,7914.0
4,2022-01-01 01:00:00+01:00,7843.0


In [5]:
# Check shape
print(load_df.shape)

(210336, 2)


In [6]:
# Rename date column
load_df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)

# Convert date column to datetime type and set time in CET
load_df['date'] = pd.to_datetime(load_df['date'], utc=True).dt.tz_convert("CET")

In [7]:
# Set date as index
load_df.set_index('date', inplace=True)

In [8]:
# Sort index from old to recent
load_df = load_df.sort_index()

In [9]:
# Convert other columns to float
cols = load_df.columns
load_df[cols] = load_df[cols].apply(pd.to_numeric, errors='coerce')

In [10]:
# Set per hour
load_df = load_df.resample('H').mean()

In [11]:
# Check data types
print(load_df.dtypes)

Actual Load    float64
dtype: object


In [12]:
# Check
load_df.head(5)

,Actual Load
date,
2017-01-01 00:00:00+01:00,10108.75
2017-01-01 01:00:00+01:00,9771.50
2017-01-01 02:00:00+01:00,9354.00
2017-01-01 03:00:00+01:00,8979.75
2017-01-01 04:00:00+01:00,8676.00


- **date**: date and time of generation (covers the next hour)
- **Actual Load**: load on the grid at a specific time (MW)

## Scheduled exchanges

In [57]:
# Inspect
exchanges_NL_BE_df.head(5)

,Unnamed: 0,scheduled exchange NL --> BE
0,2022-01-01 00:00:00+01:00,365.0
1,2022-01-01 00:15:00+01:00,358.0
2,2022-01-01 00:30:00+01:00,358.0
3,2022-01-01 00:45:00+01:00,359.0
4,2022-01-01 01:00:00+01:00,449.0


In [58]:
exchanges_FR_BE_df.head(5)

,Unnamed: 0,scheduled exchange FR --> BE
0,2022-01-01 00:00:00+01:00,4.0
1,2022-01-01 01:00:00+01:00,8.0
2,2022-01-01 02:00:00+01:00,0.0
3,2022-01-01 03:00:00+01:00,0.0
4,2022-01-01 04:00:00+01:00,0.0


In [59]:
# Check shape
print(exchanges_DE_BE_df.shape)
print(exchanges_FR_BE_df.shape)
print(exchanges_NL_BE_df.shape)
print(exchanges_GB_BE_df.shape)

(74304, 2)
(52584, 2)
(109605, 2)
(27384, 2)


In [60]:
# Rename date column
exchanges_DE_BE_df.rename(columns={'Unnamed: 0':'date'}, inplace=True)
exchanges_FR_BE_df.rename(columns={'Unnamed: 0':'date'}, inplace=True)
exchanges_NL_BE_df.rename(columns={'Unnamed: 0':'date'}, inplace=True)
exchanges_GB_BE_df.rename(columns={'Unnamed: 0':'date'}, inplace=True)

In [61]:
# Convert date column to datetime type and set time in CET
exchanges_DE_BE_df['date'] = pd.to_datetime(exchanges_DE_BE_df['date'], utc=True).dt.tz_convert("CET")
exchanges_FR_BE_df['date'] = pd.to_datetime(exchanges_FR_BE_df['date'], utc=True).dt.tz_convert("CET")
exchanges_NL_BE_df['date'] = pd.to_datetime(exchanges_NL_BE_df['date'], utc=True).dt.tz_convert("CET")
exchanges_GB_BE_df['date'] = pd.to_datetime(exchanges_GB_BE_df['date'], utc=True).dt.tz_convert("CET")

In [62]:
# Set NL and DE dataframe per hour
exchanges_DE_BE_df = exchanges_DE_BE_df.resample('H', on='date').sum().reset_index()
exchanges_NL_BE_df = exchanges_NL_BE_df.resample('H', on='date').sum().reset_index()

In [63]:
# Check shape
print(exchanges_DE_BE_df.shape)
print(exchanges_FR_BE_df.shape)
print(exchanges_NL_BE_df.shape)
print(exchanges_GB_BE_df.shape)

(18576, 2)
(52584, 2)
(52584, 2)
(27384, 2)


In [64]:
# Check
exchanges_NL_BE_df.head(5)

,date,scheduled exchange NL --> BE
0,2017-01-01 00:00:00+01:00,1904.0
1,2017-01-01 01:00:00+01:00,1542.0
2,2017-01-01 02:00:00+01:00,1707.0
3,2017-01-01 03:00:00+01:00,1730.0
4,2017-01-01 04:00:00+01:00,1885.0


In [65]:
# Join all dataframes
import_exchanges_df = exchanges_DE_BE_df.merge(exchanges_NL_BE_df, on='date', how='outer') \
                                        .merge(exchanges_FR_BE_df, on='date', how='outer') \
                                        .merge(exchanges_GB_BE_df, on='date', how='outer')

In [66]:
# Set date as index
import_exchanges_df.set_index('date', inplace=True)

In [67]:
# Sort on index
import_exchanges_df = import_exchanges_df.sort_index()

In [68]:
# Get data types
print(import_exchanges_df.dtypes)

scheduled exchange DE_LU --> BE    float64
scheduled exchange NL --> BE       float64
scheduled exchange FR --> BE       float64
scheduled exchange GB --> BE       float64
dtype: object


In [69]:
# Check
import_exchanges_df.head(5)

,scheduled exchange DE_LU --> BE,scheduled exchange NL --> BE,scheduled exchange FR --> BE,scheduled exchange GB --> BE
date,,,,
2017-01-01 00:00:00+01:00,NaN,1904.0,16.0,NaN
2017-01-01 01:00:00+01:00,NaN,1542.0,0.0,NaN
2017-01-01 02:00:00+01:00,NaN,1707.0,50.0,NaN
2017-01-01 03:00:00+01:00,NaN,1730.0,24.0,NaN
2017-01-01 04:00:00+01:00,NaN,1885.0,56.0,NaN


- **date**: date of scheduled exchange (covers next hour)
- **scheduled exchange**: the scheduled exchange (MW) from country x to country y

## Outages

In [70]:
# Inspect
outages_df.head(5)

,created_doc_time,avail_qty,biddingzone_domain,businesstype,curvetype,docstatus,end,mrid,nominal_power,plant_type,production_resource_id,production_resource_location,production_resource_name,pstn,qty_uom,resolution,revision,start
0,2020-06-08 17:38:11+02:00,0.0,BE,Planned maintenance,A03,Cancelled,2022-11-28 00:00:00+01:00,Xu7lX6yUCVjsS2nt6ZEnNw,158.0,Hydro Pumped Storage,22WCOOXIX000067T,Belgium,COO I T,1,MAW,PT60M,2,2022-02-28 00:00:00+01:00
1,2020-06-08 17:38:12+02:00,0.0,BE,Planned maintenance,A03,Cancelled,2022-04-25 18:00:00+02:00,1VZNNrKhuuugwwL9_aBO4Q,158.0,Hydro Pumped Storage,22WCOOXIX000067T,Belgium,COO I T,1,MAW,PT60M,2,2022-04-11 08:00:00+02:00
2,2020-06-08 17:38:15+02:00,0.0,BE,Planned maintenance,A03,Cancelled,2022-05-06 18:00:00+02:00,j508YN2Cl1o2CY8CY-VbMg,158.0,Hydro Pumped Storage,22WCOOXIX000067T,Belgium,COO I T,1,MAW,PT60M,2,2022-04-15 08:00:00+02:00
3,2020-06-11 10:41:35+02:00,0.0,BE,Planned maintenance,A03,Cancelled,2022-10-01 00:00:00+02:00,BJRNUsGY08yNFXZAOkKmLA,167.0,Fossil Gas,22WHERDER0001288,Belgium,HERDERSBRUG STEG,1,MAW,PT60M,2,2022-04-01 00:00:00+02:00
4,2020-07-06 14:45:03+02:00,0.0,BE,Planned maintenance,A03,Cancelled,2022-06-17 00:00:00+02:00,_c3BsiQha0-irG-Nww03Dw,156.3,Fossil Gas,22WHERDER0001288,Belgium,HERDERSBRUG STEG,1,MAW,PT60M,2,2022-06-13 00:00:00+02:00


In [71]:
# Check shape
print(outages_df.shape)

(2463, 18)


In [72]:
# Check types
print(outages_df.dtypes)

created_doc_time                 object
avail_qty                       float64
biddingzone_domain               object
businesstype                     object
curvetype                        object
docstatus                        object
end                              object
mrid                             object
nominal_power                   float64
plant_type                       object
production_resource_id           object
production_resource_location     object
production_resource_name         object
pstn                              int64
qty_uom                          object
resolution                       object
revision                          int64
start                            object
dtype: object


In [73]:
# Set date_doc_created, start and end as datetime
outages_df['created_doc_time'] = pd.to_datetime(outages_df['created_doc_time'], utc=True).dt.tz_convert("CET")
outages_df['start'] = pd.to_datetime(outages_df['start'], utc=True).dt.tz_convert("CET")
outages_df['end'] = pd.to_datetime(outages_df['end'], utc=True).dt.tz_convert("CET")

In [74]:
# Sort df on start date
outages_df = outages_df.sort_values(by='start')

In [75]:
# Reset index
outages_df = outages_df.reset_index(drop=True)

In [76]:
# Check data types
print(outages_df.dtypes)

created_doc_time                datetime64[ns, CET]
avail_qty                                   float64
biddingzone_domain                           object
businesstype                                 object
curvetype                                    object
docstatus                                    object
end                             datetime64[ns, CET]
mrid                                         object
nominal_power                               float64
plant_type                                   object
production_resource_id                       object
production_resource_location                 object
production_resource_name                     object
pstn                                          int64
qty_uom                                      object
resolution                                   object
revision                                      int64
start                           datetime64[ns, CET]
dtype: object


In [77]:
# Check
outages_df.head(5)

,created_doc_time,avail_qty,biddingzone_domain,businesstype,curvetype,docstatus,end,mrid,nominal_power,plant_type,production_resource_id,production_resource_location,production_resource_name,pstn,qty_uom,resolution,revision,start
0,2015-12-08 00:37:57+01:00,0.0,BE,Planned maintenance,A03,Cancelled,2017-01-01 00:00:00+01:00,h8RjhoLOedNVvRpOMccqSw,235.0,Fossil Hard coal,22WLANGER0001694,Belgium,LANGERLO 1,1,MAW,PT1M,6,2016-04-01 00:00:00+02:00
1,2015-12-08 00:38:05+01:00,0.0,BE,Planned maintenance,A03,Cancelled,2017-01-01 00:00:00+01:00,VfuEl2r3CHKBFDWo2Syg8g,235.0,Fossil Hard coal,22WLANGER000171H,Belgium,LANGERLO 2,1,MAW,PT1M,4,2016-04-01 00:00:00+02:00
2,2016-03-31 13:27:53+02:00,0.0,BE,Planned maintenance,A03,Cancelled,2017-01-01 00:00:00+01:00,SPIZgIxq0xN4tThO_ET_vw,235.0,Fossil Hard coal,22WLANGER0001694,Belgium,LANGERLO 1,1,MAW,PT1M,2,2016-04-01 00:00:00+02:00
3,2016-03-31 13:31:51+02:00,0.0,BE,Planned maintenance,A03,Cancelled,2017-01-01 00:00:00+01:00,zhf66iexFg9znOkkvxbq7w,235.0,Fossil Hard coal,22WLANGER000171H,Belgium,LANGERLO 2,1,MAW,PT1M,2,2016-04-01 00:00:00+02:00
4,2016-09-30 17:26:07+02:00,0.0,BE,Planned maintenance,A03,Cancelled,2017-01-01 00:00:00+01:00,zWfvRhO64Vk6TLWw3HukoQ,419.8,Fossil Gas,22WT-POWE000244W,Belgium,T-power Beringen,1,MAW,PT1M,3,2016-12-16 00:00:00+01:00


- **date_doc_created**: creation of unavailability document
- **avail_qty**: quantity still available for generation
- **biddingzone_domain**: identification of bidding zone
- **businesstype**: nature of the unavailability
- **curvetype**: represents the coded identification of the curve that is described in Period and Interval class
- **docstatus**: identification of the condition of the document (cancelled - in case of planned outages - or withdrawn - to remove outage -)
- **mRID**: unique identification of the unavailability document 
- **nominal_power**: nominal power of a generation unit
- **plant_type**: production type
- **production_resource_id**: he identification of a production unit that is affected by the unavailability.
- **production_resource_location**: name of the location
- **production_resource_name**: name of generation unit
- **pstn**: relative position within a time interval
- **qty_uom**: quantity measure, MAW=Megawatts
- **resolution**: number of units of time that compose individual step within a period (PT60M = hourly, PT30M, half hourly, ...)
- **revision**: identification of the version of the document (1 = first version, increment for each retransmission)
- **start and end**: start and end date and time of the time interval covering the whole unavailability document

# 2. Data Understanding

### Column types

In [78]:
# Get variable types
print("Column types day_ahead_df:")
print(day_ahead_df.dtypes)
print("---------------------------")
print("Column types daily_prices_df:")
print(daily_prices_df.dtypes)
print("---------------------------")
print("Column types generation_df:")
print(generation_df.dtypes)
print("---------------------------")
print("Column types production_type_df:")
print(production_type_df.dtypes)
print("---------------------------")
print("Column types load_df:")
print(load_df.dtypes)
print("---------------------------")
print("Column types import_exchanges_df:")
print(import_exchanges_df.dtypes)
print("---------------------------")
print("Column types outages_df:")
print(outages_df.dtypes)
print("---------------------------")

Column types day_ahead_df:
day_ahead_price    float64
dtype: object
---------------------------
Column types daily_prices_df:
day_ahead_price    float64
gas_price          float64
dtype: object
---------------------------
Column types generation_df:
Amercoeur 1 R GT                    float64
Amercoeur 1 R ST                    float64
Belwind Phase 1                     float64
COO 1 T                             float64
COO 2 T                             float64
COO 3 T                             float64
COO 4 T                             float64
COO 5 T                             float64
COO 6 T                             float64
DOEL 1                              float64
DOEL 2                              float64
DOEL 3                              float64
DOEL 4                              float64
DROGENBOS GT1                       float64
DROGENBOS GT2                       float64
DROGENBOS ST                        float64
EDF Luminus Seraing GT1             float64
ED

### Dataframe shapes

In [79]:
# Get shapes of dataframes
print(f"Shape day_ahead_df: {day_ahead_df.shape}")
print(f"Shape daily_prices_df: {daily_prices_df.shape}")
print(f"Shape generation_df: {generation_df.shape}")
print(f"Shape production_type_df: {production_type_df.shape}")
print(f"Shape load_df: {load_df.shape}")
print(f"Shape import_exchanges_df: {import_exchanges_df.shape}")
print(f"Shape outages_df: {outages_df.shape}")

Shape day_ahead_df: (52584, 1)
Shape daily_prices_df: (1896, 2)
Shape generation_df: (52584, 47)
Shape production_type_df: (52584, 14)
Shape load_df: (52584, 1)
Shape import_exchanges_df: (52584, 4)
Shape outages_df: (2463, 18)


### Missing values

In [80]:
# Get number of missing values in each dataframe
print("Number of missing values day_ahead_df: ")
print(day_ahead_df.isnull().sum())
print("---------------------------------------")
print("Number of missing values daily_prices_df: ")
print(daily_prices_df.isnull().sum())
print("---------------------------------------")
print("Number of missing values generation_df: ")
print(generation_df.isnull().sum())
print("---------------------------------------")
print("Number of missing values production_type_df: ")
print(production_type_df.isnull().sum())
print("---------------------------------------")
print("Number of missing values load_df: ")
print(load_df.isnull().sum())
print("---------------------------------------")
print("Number of missing values import_exchanges_df: ")
print(import_exchanges_df.isnull().sum())
print("---------------------------------------")
print("Number of missing values outages_df: ")
print(outages_df.isnull().sum())

Number of missing values day_ahead_df: 
day_ahead_price    0
dtype: int64
---------------------------------------
Number of missing values daily_prices_df: 
day_ahead_price    0
gas_price          0
dtype: int64
---------------------------------------
Number of missing values generation_df: 
Amercoeur 1 R GT                        0
Amercoeur 1 R ST                        0
Belwind Phase 1                         0
COO 1 T                                 0
COO 2 T                                 3
COO 3 T                                 0
COO 4 T                                 0
COO 5 T                                 0
COO 6 T                                 0
DOEL 1                                  0
DOEL 2                                  0
DOEL 3                               2209
DOEL 4                                  0
DROGENBOS GT1                           0
DROGENBOS GT2                           0
DROGENBOS ST                            1
EDF Luminus Seraing GT1            

In [81]:
# Check missing values for DOEL 3
generation_df[generation_df['DOEL 3'].isnull() == True][['DOEL 3']]

,DOEL 3
date,
2022-10-01 00:00:00+02:00,NaN
2022-10-01 01:00:00+02:00,NaN
2022-10-01 02:00:00+02:00,NaN
2022-10-01 03:00:00+02:00,NaN
2022-10-01 04:00:00+02:00,NaN
...,...
2022-12-31 19:00:00+01:00,NaN
2022-12-31 20:00:00+01:00,NaN
2022-12-31 21:00:00+01:00,NaN


In [82]:
# Check missing values for DOEL 3
generation_df[generation_df.index >= '2022-09-30'][['DOEL 3']].head(50)

,DOEL 3
date,
2022-09-30 00:00:00+02:00,0.0
2022-09-30 01:00:00+02:00,0.0
2022-09-30 02:00:00+02:00,0.0
2022-09-30 03:00:00+02:00,0.0
2022-09-30 04:00:00+02:00,0.0
2022-09-30 05:00:00+02:00,0.0
2022-09-30 06:00:00+02:00,0.0
2022-09-30 07:00:00+02:00,0.0
2022-09-30 08:00:00+02:00,0.0


DOEL 3 was forced to close in accordance with the nuclear exit law
https://nuclear.engie-electrabel.be/nl/kernenergie/definitieve-stop-van-doel-3
Officialy, closing date was 1/10, therefore all values after that date are NaN. In practice, DOEL 3 stopped producing 23/09 because of the regulatory limitation of the duration of 1 particle cycle to a maximum of 365 days.

# 3. Data Cleaning and feature engineering

In [83]:
# Create new columns in outages
outages_df['year_outage'] = outages_df['start'].dt.year

In [84]:
# Only keep relevant columns for outages
outages_df = outages_df[['start', 'end', 'production_resource_name', 'plant_type', 'avail_qty', 'nominal_power', 'businesstype', 'year_outage']]

In [85]:
# Create extra column 'outage_time' in outages_df
outages_df['outage_time'] = outages_df['end'] - outages_df['start']

In [86]:
# Delete columns in production_type_df: Hydro Pumped Storage.1 & Wind Offshore.1
production_type_df = production_type_df.drop(['Hydro Pumped Storage.1', 'Wind Offshore.1'], axis=1)

In [87]:
# Impute missing values
generation_df = generation_df.fillna(0)
production_type_df = production_type_df.fillna(0)
import_exchanges_df = import_exchanges_df.fillna(0)

In [88]:
# Check
print(generation_df.isnull().sum().sum())
print(production_type_df.isnull().sum().sum())
print(import_exchanges_df.isnull().sum().sum())

0
0
0


In [89]:
# Check 
outages_df.head(5)

,start,end,production_resource_name,plant_type,avail_qty,nominal_power,businesstype,year_outage,outage_time
0,2016-04-01 00:00:00+02:00,2017-01-01 00:00:00+01:00,LANGERLO 1,Fossil Hard coal,0.0,235.0,Planned maintenance,2016,275 days 01:00:00
1,2016-04-01 00:00:00+02:00,2017-01-01 00:00:00+01:00,LANGERLO 2,Fossil Hard coal,0.0,235.0,Planned maintenance,2016,275 days 01:00:00
2,2016-04-01 00:00:00+02:00,2017-01-01 00:00:00+01:00,LANGERLO 1,Fossil Hard coal,0.0,235.0,Planned maintenance,2016,275 days 01:00:00
3,2016-04-01 00:00:00+02:00,2017-01-01 00:00:00+01:00,LANGERLO 2,Fossil Hard coal,0.0,235.0,Planned maintenance,2016,275 days 01:00:00
4,2016-12-16 00:00:00+01:00,2017-01-01 00:00:00+01:00,T-power Beringen,Fossil Gas,0.0,419.8,Planned maintenance,2016,16 days 00:00:00


# 4. Write to csv

In [90]:
# Copy latest dataframes
day_ahead_final = day_ahead_df
daily_prices_final = daily_prices_df
generation_final = generation_df
production_type_final = production_type_df
load_final = load_df
import_exchanges_final = import_exchanges_df
outages_final = outages_df

In [91]:
# Write final dataframes to csv
day_ahead_final.to_csv('.\Data\Final\day_ahead_final.csv', sep=';')
daily_prices_final.to_csv('.\Data\Final\daily_prices_final.csv', sep=';')
generation_final.to_csv('.\Data\Final\generation_final.csv', sep=';')
production_type_final.to_csv('.\Data\Final\production_type_final.csv', sep=';')
load_final.to_csv('.\Data\Final\load_final.csv', sep=';')
import_exchanges_final.to_csv('.\Data\Final\import_final.csv', sep=';')
outages_final.to_csv('.\Data\Final\outages_final.csv', sep=';')